## ResNet

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import InceptionResNetV2
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D, Dropout, Dense
from keras.regularizers import l2
from keras.callbacks import LearningRateScheduler
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np

number_of_classes = 7

# Chemins des répertoires de données d'entraînement et de test
train_data_dir = r"/content/drive/MyDrive/Fil_rouge/Dataset/archive(3)/DATASET/train"
test_data_dir = r"/content/drive/MyDrive/Fil_rouge/Dataset/archive(3)/DATASET/test"

# Prétraitement des données d'entraînement
train_datagen = ImageDataGenerator(
    rescale=1./255,          # Redimensionner les valeurs des pixels à [0, 1]
    rotation_range=15,       # Rotation aléatoire des images de 15 degrés
    width_shift_range=0.1,   # Décalage horizontal aléatoire jusqu'à 10%
    height_shift_range=0.1,  # Décalage vertical aléatoire jusqu'à 10%
    shear_range=0.1,         # Cisaillement aléatoire jusqu'à 10%
    zoom_range=0.1,          # Zoom aléatoire jusqu'à 10%
    horizontal_flip=True,    # Flip horizontal aléatoire
    fill_mode='nearest'      # Remplir les pixels vides avec la valeur du pixel le plus proche
)

# Prétraitement des données de test
test_datagen = ImageDataGenerator(rescale=1./255)  # Redimensionner les valeurs des pixels à [0, 1]

# Générateur de données d'entraînement
train_generator = train_datagen.flow_from_directory(
    train_data_dir,         # Répertoire contenant les images d'entraînement
    target_size=(100, 100), # Redimensionner toutes les images à 100x100 pixels
    batch_size=32,          # Traiter les images par lots de 32
    class_mode='categorical'# Les étiquettes des classes sont codées de manière catégorielle
)

# Générateur de données de test
test_generator = test_datagen.flow_from_directory(
    test_data_dir,          # Répertoire contenant les images de test
    target_size=(100, 100), # Redimensionner toutes les images à 100x100 pixels
    batch_size=32,          # Traiter les images par lots de 32
    class_mode='categorical', # Les étiquettes des classes sont codées de manière catégorielle
    shuffle=False           # Ne pas mélanger les images lors de la génération de lots
)

# Chargement du modèle de base InceptionResNetV2 pré-entraîné
base_model = InceptionResNetV2(include_top=False, input_shape=(100, 100, 3))

# Construction du modèle séquentiel
model = Sequential()
model.add(base_model)                  # Ajouter le modèle de base
model.add(GlobalAveragePooling2D())    # Appliquer un pool moyen global 2D
model.add(Dropout(0.5))                # Appliquer un dropout de 50% pour régularisation
model.add(Dense(256, activation='relu', kernel_regularizer=l2(0.01)))  # Couche dense avec régularisation L2
model.add(Dense(number_of_classes, activation='softmax'))  # Couche de sortie pour classification

# Compilation du modèle
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Geler les couches du modèle de base sauf les 10 dernières
for layer in base_model.layers[:-10]:
    layer.trainable = False

# Définition du scheduler d'apprentissage
def lr_scheduler(epoch, lr):
    return lr * 0.95

# Callback pour ajuster le taux d'apprentissage
lr_callback = LearningRateScheduler(lr_scheduler)

# Entraînement du modèle
history = model.fit(
    train_generator,                  # Générateur de données d'entraînement
    steps_per_epoch=len(train_generator),  # Nombre de pas par époque
    epochs=40,                        # Nombre d'époques d'entraînement
    validation_data=test_generator,    # Générateur de données de validation
    validation_steps=len(test_generator), # Nombre de pas de validation
    callbacks=[lr_callback]            # Utiliser le callback pour ajuster le taux d'apprentissage
)

# Tracer les courbes de précision et de perte
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

# Prédictions sur les données de test
y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = test_generator.classes

# Générer la matrice de confusion
cm = confusion_matrix(y_true, y_pred_classes)
cmd = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=test_generator.class_indices)
cmd.plot(cmap=plt.cm.Blues)
plt.show()
